In [21]:
import pandas as pd
import pickle
import math
import datetime as dt


def calculo_nomina(salario_actual, años_hasta_jubilación, ipc):
    redondeado = math.floor(años_hasta_jubilación)
    salario = salario_actual
    for i in range(0, redondeado):
        salario = salario + salario * ipc.iloc[i, 1]
    return float(salario)

def calcular_monto_mensual(capital, intereses):
    n_periodos = len(intereses)
    sumatorio = sum((1 + intereses[i]) ** (i + 1) for i in range(n_periodos))
    if sumatorio == 0:
        return None
    return capital / sumatorio

def calcular_primas_jubilacion_df(df, ipc, interes1, interes2, duracion_interes1, interes_rendimiento1, interes_rendimiento2, duracion_interes_rendimiento1):
    resultados = []
    with open("notebooks/Data Mining/modelo.pkl", "rb") as f:
        modelo = pickle.load(f)
    X_2 = df.drop(['ID', 'FECHA NAC', 'SEXO', 'FECHA ENTRADA', 'PARA CONTAR MESES'], axis=1)
    X_2.columns = ['Ingreso Anual (€)','Edad']
    X_2 = X_2[['Edad','Ingreso Anual (€)']] 
    df["AÑOS HASTA JUBILACION"] = modelo.predict(X_2)
    for _, row in df.iterrows():
        salario_final = calculo_nomina(row['NOMINA BRUTA 01/01/2025'], row['AÑOS HASTA JUBILACION'], ipc)
        porcentaje_renta = min((row['EDAD'] // 4) * 0.0225, 0.19)
        m1 = (salario_final * porcentaje_renta) / 12
        
        fecha_jubilacion = dt.datetime(2025, 1, 1) + dt.timedelta(days=row['AÑOS HASTA JUBILACION'] * 365)
        fecha_jubilacion = fecha_jubilacion.strftime('%Y-%m-%d')
        date_range = pd.date_range(start=fecha_jubilacion, periods=22 * 12, freq='MS')
        date_list = date_range.strftime('%Y-%m').tolist()
        
        int1 = ((1 + (interes1 * 0.01)) ** (1 / 12)) - 1
        int2 = ((1 + (interes2 * 0.01)) ** (1 / 12)) - 1
        año1, mes1 = duracion_interes1
        
        intereses = [int1] * (año1 * 12 + mes1) + [int2] * ((22 * 12) - (año1 * 12 + mes1))
        
        rentas = []
        valor = m1
        for fecha in date_list:
            if fecha[-2:] == '01':
                valor *= 1.03
            rentas.append(valor)
        
        df_rentas = pd.DataFrame({'Fecha': date_list, 'Intereses': intereses, 'Pagos': rentas})
        
        capital_jubilacion = 0
        for i in range(len(df_rentas) - 1, -1, -1):
            renta = float(df_rentas.iloc[i, 2]) * (1 + float(df_rentas.iloc[i, 1])) ** -1
            for interes in range(i, -1, -1):
                renta = renta * (1 + float(df_rentas.iloc[interes, 1])) ** -1
            capital_jubilacion += renta
        
        fechas = pd.date_range(start='2025-01-01', end=fecha_jubilacion, freq='MS')
        fechas_formateadas = fechas.strftime('%Y-%m')
        
        int_ren1 = ((1 + (interes_rendimiento1 * 0.01)) ** (1 / 12)) - 1
        int_ren2 = ((1 + (interes_rendimiento2 * 0.01)) ** (1 / 12)) - 1
        año2, mes2 = duracion_interes_rendimiento1
        
        intereses1 = [int_ren1] * (año2 * 12 + mes2) + [int_ren2] * (len(fechas_formateadas) - (año2 * 12 + mes2))
        df1 = pd.DataFrame({'Fecha': fechas_formateadas, 'Intereses': intereses1})
        
        capital_actual = capital_jubilacion
        for i in range(len(df1) - 1, -1, -1):
            capital_actual = capital_actual * (1 + float(df1.iloc[i, 1])) ** -1
        
        monto_mensual = calcular_monto_mensual(capital_jubilacion, df1['Intereses'].tolist())
        
        resultados.append([row['ID'], capital_jubilacion, capital_actual, monto_mensual])
    
    return pd.DataFrame(resultados, columns=['ID', 'Capital Jubilación', 'Capital Actual', 'Monto Mensual'])

# Cargar los datos y calcular resultados
df_input = pd.read_csv('Datos/Limpios/datos1_limpios.csv', index_col=0)
ipc_data = pd.read_excel('Datos/Originales/Dataset_1.xlsx', sheet_name='IPC')
resultados_df = calcular_primas_jubilacion_df(df_input, ipc_data, 8.9, 8.9, [6,9], 8.9, 8.9, [7,0])
print(resultados_df)


     ID  Capital Jubilación  Capital Actual  Monto Mensual
0     1        53789.500404    16304.489071     165.641224
1     2        84724.660895     4195.435782      31.250342
2     3        62036.917812    22619.706673     252.041851
3     4        45331.778918    13546.937657     136.787124
4     5        72865.257000    19435.170941     187.648184
..  ...                 ...             ...            ...
95   96        88130.282948    26524.621249     268.642736
96   97        85621.983772     6539.997462      50.131035
97   98        90936.311623     3201.791555      23.495314
98   99       120854.087805    10792.898304      83.904760
99  100       108397.470607    14721.417541     120.603960

[100 rows x 4 columns]


In [22]:
resultados_df['Capital Actual'].sum()

np.float64(1262744.7370012922)

In [23]:
resultados_df['Capital Jubilación'].sum()

np.float64(8747629.049988018)

In [24]:
resultados_df['Monto Mensual'].sum()

np.float64(11913.79834826541)

In [26]:
import pandas as pd
import pickle
import math
import datetime as dt

def calculo_nomina(salario_actual, años_hasta_jubilación, ipc):
    redondeado = math.floor(años_hasta_jubilación)
    salario = salario_actual
    for i in range(0, redondeado):
        salario = salario + salario * ipc.iloc[i, 1]
    return float(salario)

def calcular_monto_mensual(capital, intereses):
    n_periodos = len(intereses)
    sumatorio = sum((1 + intereses[i]) ** (i + 1) for i in range(n_periodos))
    return capital / sumatorio

def calcular_primas_jubilacion_df(df, ipc, lista_intereses):
    resultados = []
    with open("notebooks/Data Mining/modelo.pkl", "rb") as f:
        modelo = pickle.load(f)
    X_2 = df.drop(['ID', 'FECHA NAC', 'SEXO', 'FECHA ENTRADA', 'PARA CONTAR MESES'], axis=1)
    X_2.columns = ['Ingreso Anual (€)','Edad']
    X_2 = X_2[['Edad','Ingreso Anual (€)']]
    df["AÑOS HASTA JUBILACION"] = modelo.predict(X_2)
    for _, row in df.iterrows():
        salario_final = calculo_nomina(row['NOMINA BRUTA 01/01/2025'], row['AÑOS HASTA JUBILACION'], ipc)
        porcentaje_renta = min((row['EDAD'] // 4) * 0.0225, 0.19)
        m1 = (salario_final * porcentaje_renta) / 12
        
        fecha_jubilacion = dt.datetime(2025, 1, 1) + dt.timedelta(days=row['AÑOS HASTA JUBILACION'] * 365)
        fecha_jubilacion = fecha_jubilacion.strftime('%Y-%m-%d')
        date_range = pd.date_range(start=fecha_jubilacion, periods=22 * 12, freq='MS')
        date_list = date_range.strftime('%Y-%m').tolist()
        
        if len(lista_intereses) < len(date_list):
            raise ValueError(f"La lista de intereses es demasiado corta. Se requieren {len(date_list)} valores, pero solo se proporcionaron {len(lista_intereses)}.")
        
        intereses = lista_intereses[:len(date_list)]
        
        rentas = []
        valor = m1
        for fecha in date_list:
            if fecha[-2:] == '01':
                valor *= 1.03
            rentas.append(valor)
        
        df_rentas = pd.DataFrame({'Fecha': date_list, 'Intereses': intereses, 'Pagos': rentas})
        
        capital_jubilacion = 0
        for i in range(len(df_rentas) - 1, -1, -1):
            renta = float(df_rentas.iloc[i, 2]) * (1 + float(df_rentas.iloc[i, 1])) ** -1
            for interes in range(i, -1, -1):
                renta = renta * (1 + float(df_rentas.iloc[interes, 1])) ** -1
            capital_jubilacion += renta
        
        fechas = pd.date_range(start='2025-01-01', end=fecha_jubilacion, freq='MS')
        fechas_formateadas = fechas.strftime('%Y-%m')
        
        if len(lista_intereses) < len(fechas_formateadas):
            raise ValueError(f"La lista de intereses es demasiado corta. Se requieren {len(fechas_formateadas)} valores, pero solo se proporcionaron {len(lista_intereses)}.")
        
        intereses1 = lista_intereses[:len(fechas_formateadas)]
        df1 = pd.DataFrame({'Fecha': fechas_formateadas, 'Intereses': intereses1})
        
        capital_actual = capital_jubilacion
        for i in range(len(df1) - 1, -1, -1):
            capital_actual = capital_actual * (1 + float(df1.iloc[i, 1])) ** -1
        
        monto_mensual = calcular_monto_mensual(capital_jubilacion, df1['Intereses'].tolist())
        
        resultados.append([row['ID'], capital_jubilacion, capital_actual, monto_mensual])
    
    return pd.DataFrame(resultados, columns=['ID', 'Capital Jubilación', 'Capital Actual', 'Monto Mensual'])


In [27]:
df_input = pd.read_csv('Datos/Limpios/datos1_limpios.csv', index_col=0)
ipc_data = pd.read_excel('Datos/Originales/Dataset_1.xlsx', sheet_name='IPC')
intereses_lista = pd.read_csv('Datos/Limpios/datos1_limpios.csv', index_col=0)
intereses_lista = pd.read_csv('Datos/Intereses/Datos históricos del S&P 500.csv', index_col=0)
lista=list(intereses_lista['% var.'])
intereses_lista = [float(str(i).replace('%', '').replace(',', '.')) / 100 for i in lista]

resultados_df = calcular_primas_jubilacion_df(df_input, ipc_data, intereses_lista)
print(resultados_df)


     ID  Capital Jubilación  Capital Actual  Monto Mensual
0     1        48186.160343    11271.687573   3.062069e-03
1     2        75895.912642     4731.424973   3.043940e-14
2     3        55571.855778    15987.780240   3.692455e+00
3     4        40607.920704     9214.582508   2.580462e-03
4     5        65274.938021    12174.246245   3.549037e-03
..  ...                 ...             ...            ...
95   96        78947.271704    18487.650995   5.016834e-03
96   97        76699.631264     6365.166951   5.613758e-08
97   98        81460.277209     3035.687248   1.771217e-20
98   99       108254.498618    14456.320585   8.101126e-08
99  100        97099.907430    18153.115842   1.394233e-05

[100 rows x 4 columns]


In [28]:
resultados_df['Capital Actual'].sum()

np.float64(1195478.4404291294)

In [29]:
resultados_df['Capital Jubilación'].sum()

np.float64(7835991.299050733)

In [30]:
resultados_df['Monto Mensual'].sum()

np.float64(82.90849485047693)